In [1]:
# ============================================================
# 0) INSTALLS (run once)
# ============================================================
!pip install firecrawl-py groq python-dotenv tqdm beautifulsoup4 lxml

In [2]:
# ============================================================
# 1) SETUP: ENV + CLIENTS
# ============================================================
import os, re, json, ast
from typing import Any, Dict, List
from dotenv import load_dotenv
from tqdm import tqdm

load_dotenv()

FIRECRAWL_API_KEY = os.getenv("FIRECRAWL_API_KEY")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

print("Firecrawl key loaded:", bool(FIRECRAWL_API_KEY))
print("Groq key loaded:", bool(GROQ_API_KEY))

from groq import Groq
groq_client = Groq(api_key=GROQ_API_KEY)

Firecrawl key loaded: True
Groq key loaded: True


In [3]:
# Pick a Groq model. If rate-limited, try llama-3.1-70b-versatile
MODEL = "llama-3.3-70b-versatile"

from firecrawl import FirecrawlApp
firecrawl = FirecrawlApp(api_key=FIRECRAWL_API_KEY)

from bs4 import BeautifulSoup


In [4]:
# ============================================================
# 2) GENERIC HELPERS
# ============================================================

def groq_chat(prompt: str, model: str = MODEL, temperature: float = 0.2) -> str:
    """Small wrapper for Groq chat."""
    resp = groq_client.chat.completions.create(
        model=model,
        temperature=temperature,
        messages=[{"role": "user", "content": prompt}],
    )
    return resp.choices[0].message.content

def extract_first_json(text: str) -> Dict[str, Any]:
    """Extract JSON object from model output (even if wrapped with extra text)."""
    text = text.strip()
    try:
        return json.loads(text)
    except:
        pass
    m = re.search(r"\{.*\}", text, flags=re.DOTALL)
    if not m:
        raise ValueError("Could not extract JSON from model output.")
    return json.loads(m.group(0))

def parse_url_list(model_text: str) -> List[str]:
    """
    Parse a Python list of URLs from model output.
    Handles fenced blocks like ```python ... ``` and other extra text.
    Uses ast.literal_eval (safe).
    """
    s = model_text.strip()
    s = re.sub(r"^```(?:python)?\s*", "", s)
    s = re.sub(r"\s*```$", "", s)

    m = re.search(r"\[.*\]", s, flags=re.DOTALL)
    if m:
        s = m.group(0)

    try:
        data = ast.literal_eval(s)
        if isinstance(data, list):
            return [u.strip() for u in data if isinstance(u, str) and u.strip().startswith("http")]
    except Exception as e:
        print("parse_url_list failed:", e)
        print("raw sample:", model_text[:400])
    return []

# Firecrawl v2 Document/dict extractors
def extract_markdown(scrape_result) -> str:
    if scrape_result is None:
        return ""
    if hasattr(scrape_result, "markdown") and scrape_result.markdown:
        return scrape_result.markdown
    if isinstance(scrape_result, dict):
        return scrape_result.get("markdown", "") or ""
    if hasattr(scrape_result, "data"):
        data = scrape_result.data
        if isinstance(data, dict):
            return data.get("markdown", "") or ""
        if hasattr(data, "markdown"):
            return data.markdown or ""
    return ""

def extract_html(scrape_result) -> str:
    if scrape_result is None:
        return ""
    if hasattr(scrape_result, "html") and scrape_result.html:
        return scrape_result.html
    if isinstance(scrape_result, dict):
        return scrape_result.get("html", "") or ""
    if hasattr(scrape_result, "data"):
        data = scrape_result.data
        if isinstance(data, dict):
            return data.get("html", "") or ""
        if hasattr(data, "html"):
            return data.html or ""
    return ""

In [5]:
# ============================================================
# 3) URL GENERATORS (Groq)
#    - avoids Firecrawl search credits entirely
# ============================================================

def generate_movie_urls(movie_name: str, max_urls: int = 5) -> List[str]:
    """
    Generate URLs to research a movie (wiki + analysis pages).
    """
    prompt = f"""
You are gathering research sources for film analysis.

Movie: {movie_name}

Return up to {max_urls} high-quality URLs:
- Wikipedia page (preferably)
- 2–3 analysis/review pages
- avoid streaming links
- avoid spam

Return ONLY a Python list of URLs.
"""
    out = groq_chat(prompt, temperature=0)
    return parse_url_list(out)



In [6]:
# ============================================================
# 4) SCRAPING FUNCTIONS (Firecrawl v2)
# ============================================================

def scrape_urls_to_bundle(urls: List[str], max_pages: int = 3, per_source_chars: int = 2500, total_chars: int = 9000) -> str:
    """
    Scrape up to max_pages URLs and return a condensed bundle (markdown).
    Used for seed movie research.
    """
    chunks = []
    for url in urls[:max_pages]:
        try:
            res = firecrawl.scrape(url=url, formats=["markdown"])
            md = extract_markdown(res)
            if not md:
                continue
            chunks.append(f"SOURCE: {url}\n{md[:per_source_chars]}")
        except Exception as e:
            print("Scrape error:", url, e)
    bundle = "\n\n".join(chunks)
    return bundle[:total_chars]

def research_movie(movie: str, max_pages: int = 3) -> str:
    """
    Movie research: Groq → URLs → Firecrawl scrape → text bundle.
    """
    print(f"\n🎬 Researching: {movie}")
    urls = generate_movie_urls(movie, max_urls=5)
    print("URLs found:", len(urls))
    for u in urls[:max_pages]:
        print(" -", u)

    if not urls:
        return ""
    text = scrape_urls_to_bundle(urls, max_pages=max_pages)
    print("Research length:", len(text))
    return text

def research_movies(movie_list: List[str], max_pages_per_movie: int = 3) -> Dict[str, str]:
    """
    Research multiple movies. Always returns dict {movie: text}.
    """
    out: Dict[str, str] = {}
    for m in tqdm(movie_list):
        out[m] = research_movie(m, max_pages=max_pages_per_movie)
    return out

In [7]:
# ============================================================
# 5) SEED MOVIE FINGERPRINT AGENT (Groq)
# ============================================================

FINGERPRINT_SCHEMA_HINT = """
Return ONLY JSON with these keys:

authorship_voice: {
  director_style: [strings],
  screenwriter_style: [strings],
  cinematography_style: [strings],
  editing_style: [strings]
},

narrative_architecture: {
  structure_type: string,
  inciting_incident_timing: "early"|"mid"|"late"|null,
  pacing: "slow"|"moderate"|"fast"|null,
  ending_type: "ambiguous"|"resolved"|"ironic"|"circular"|null,
  conflict_type: "internal"|"external"|"mixed"|null
},

themes_subtext: {
  primary_themes: [strings],
  motifs: [strings],
  worldview: "bleak"|"hopeful"|"mixed"|null,
  moral_stance: "compassionate"|"cynical"|"neutral"|null
},

style_tone: {
  realism_vs_stylized: number,            // 0.0 to 1.0
  humor_style: "none"|"deadpan"|"dark"|"broad"|null,
  dread_style: "psychological"|"cosmic"|"social"|"bodily"|null,
  performance_style: "naturalistic"|"theatrical"|null
},

extras: {
  dialogue_density: "low"|"med"|"high"|null,
  narrative_mode: "linear"|"nonlinear"|"elliptical"|null,
  intensity_curve: "gradual"|"spiky"|"constant"|null
},

confidence: {
  overall: number,                        // 0.0 to 1.0
  low_confidence_fields: [strings]
},

non_spoiler_notes: [strings]              // 2–5 bullets, NO plot spoilers
"""

def fingerprint_movie(movie_title: str, research_text: str) -> Dict[str, Any]:
    prompt = f"""
You are a film student and critic. Analyze the film's craft and storytelling style.
IMPORTANT:
- Do NOT include spoilers.
- Do NOT describe specific plot events, twists, or the ending.
- You MAY label ending_type but do NOT explain what happens.

Movie: {movie_title}

Evidence (web excerpts):
{research_text}

{FINGERPRINT_SCHEMA_HINT}

ONLY output JSON.
"""
    out = groq_chat(prompt, temperature=0.2)
    return extract_first_json(out)

def fingerprint_seed_movies(seed_movies: List[str], research_data: Dict[str, str]) -> Dict[str, Dict[str, Any]]:
    fps: Dict[str, Dict[str, Any]] = {}
    for m in tqdm(seed_movies):
        txt = research_data.get(m, "")
        if not isinstance(txt, str):
            txt = str(txt)
        if len(txt.strip()) < 500:
            print(f"⚠️ Skipping {m}: not enough research text ({len(txt)} chars)")
            continue
        fps[m] = fingerprint_movie(m, txt)
    return fps


In [8]:
# ============================================================
# 6) TASTE PROFILER AGENT (Groq)
# ============================================================

TASTE_SCHEMA_HINT = """
Return ONLY JSON with keys:

taste_summary: string,
core_signals: [strings],
secondary_signals: [strings],
avoid_signals: [strings],
query_pack: {
  anchors: [strings],
  must_have: [strings],
  should_have: [strings],
  avoid: [strings]
}
"""

def taste_profiler(seed_fingerprints: Dict[str, Dict[str, Any]]) -> Dict[str, Any]:
    prompt = f"""
You are a film student building a taste profile from the provided films.
Infer what the viewer consistently likes in:
- authorship/voice
- narrative architecture
- themes/subtext
- style/tone

Fingerprints (JSON per movie):
{json.dumps(seed_fingerprints, ensure_ascii=False)[:14000]}

{TASTE_SCHEMA_HINT}

ONLY output JSON.
"""
    out = groq_chat(prompt, temperature=0.2)
    return extract_first_json(out)

def build_search_query(taste_profile: Dict[str, Any]) -> str:
    qp = taste_profile.get("query_pack", {}) or {}
    anchors = qp.get("anchors", []) or []
    must = qp.get("must_have", []) or []
    should = qp.get("should_have", []) or []
    avoid = qp.get("avoid", []) or []

    anchor_part = " and ".join(anchors[:2]) if anchors else ""
    must_part = " ".join([f'"{x}"' for x in must[:4]])
    should_part = " ".join([f'"{x}"' for x in should[:2]])
    avoid_part = " ".join([f'-"{x}"' for x in avoid[:3]])

    query = f"movies like {anchor_part} {must_part} {should_part} {avoid_part}".strip()
    return re.sub(r"\s+", " ", query)

In [9]:
# ============================================================
# 7) CANDIDATE EXTRACTION (HTML parsing + LLM cleaning)
# ============================================================



DIRECT_CANDIDATES_SCHEMA = """
Return ONLY JSON with:
titles: [strings]  // 30 movie titles
"""

def propose_candidates_directly(taste_profile: dict, seed_movies: List[str], n: int = 30) -> List[str]:
    prompt = f"""
You are a film-student recommender. Propose {n} movie titles that match this taste profile.

Taste profile:
{json.dumps(taste_profile, ensure_ascii=False)}

Seed movies (do NOT include these):
{seed_movies}

Rules:
- Choose films that match authorship/voice, narrative structure, themes, and tone.
- Avoid spoilers.
- Prefer interesting, non-obvious picks when possible.
- Include a mix: 70% strong matches, 30% adjacent “stretch” picks.

Return ONLY JSON.

{DIRECT_CANDIDATES_SCHEMA}
"""
    out = groq_chat(prompt, temperature=0.4)
    data = extract_first_json(out)
    titles = data.get("titles", [])

    if not isinstance(titles, list):
        return []

    # Dedupe preserving order
    seen = set()
    deduped = []
    for t in titles:
        if isinstance(t, str):
            k = t.strip().lower()
            if k and k not in seen:
                seen.add(k)
                deduped.append(t.strip())
    return deduped

In [10]:

# ============================================================
# 8) CURATOR (pick 5) + EXPLANATIONS (spoiler-free)
# ============================================================

CURATOR_SCHEMA = """
Return ONLY JSON:
{
  "recommendations": [
    {
      "title": string,
      "year": string|null,
      "why_selected": [strings]
    }
  ]
}
"""

def curate_five_movies(taste_profile: Dict[str, Any], candidate_pool: List[str], seed_movies: List[str]) -> Dict[str, Any]:
    prompt = f"""
You are a film student curator. Select 5 films for the viewer.

Taste profile:
{json.dumps(taste_profile, ensure_ascii=False)}

Seed movies (do NOT recommend these):
{seed_movies}

Candidate pool:
{candidate_pool[:120]}

Rules:
- Choose 5 films that best match the taste profile.
- No ranking, just 5 picks.
- Keep reasoning spoiler-free: craft, tone, themes, narrative style.
- Avoid repeating the same explanation.

{CURATOR_SCHEMA}

ONLY output JSON.
"""
    out = groq_chat(prompt, temperature=0.2)
    return extract_first_json(out)

EXPLAIN_SCHEMA = """
Return ONLY JSON:
{
  "cards": [
    {
      "title": string,
      "year": string|null,
      "why_this_fits": [strings],   // exactly 3 bullets
      "watch_for": string
    }
  ]
}
"""

def write_explanations(taste_profile: Dict[str, Any], curated: Dict[str, Any]) -> Dict[str, Any]:
    prompt = f"""
Write spoiler-free recommendation cards.

Taste profile:
{json.dumps(taste_profile, ensure_ascii=False)}

Selected movies:
{json.dumps(curated, ensure_ascii=False)}

Rules:
- No plot spoilers, no twist mention, no ending description.
- Explain via: authorship/voice, narrative architecture, themes/subtext, style/tone.
- Each card should feel distinct and specific.
- Use exactly 3 bullets for why_this_fits.

{EXPLAIN_SCHEMA}

ONLY output JSON.
"""
    out = groq_chat(prompt, temperature=0.3)
    return extract_first_json(out)

def print_recommendations(taste_profile: Dict[str, Any], final_cards: Dict[str, Any]):
    print("\n" + "="*90)
    print("YOUR CINEMATIC TASTE (film-student summary)")
    print("="*90)
    print((taste_profile.get("taste_summary", "") or "").strip())

    core = taste_profile.get("core_signals", []) or []
    if core:
        print("\nCore signals:")
        for s in core[:8]:
            print(" -", s)

    print("\n" + "="*90)
    print("RECOMMENDATIONS (spoiler-free)")
    print("="*90)

    cards = final_cards.get("cards", []) or []
    for i, c in enumerate(cards, start=1):
        title = c.get("title", "")
        year = c.get("year")
        header = f"{i}. {title}" + (f" ({year})" if year else "")
        print("\n" + header)
        print("-" * len(header))

        for b in (c.get("why_this_fits", []) or [])[:3]:
            print(" •", b)

        wf = (c.get("watch_for", "") or "").strip()
        if wf:
            print("   Watch for:", wf)

In [11]:
# ============================================================
# 9) RUN THE FULL MVP
# ============================================================

seed_movies = [
    "The Pitt",
    "Industry",
    "Bridgerton",
    "Heated Rivalry",
    "Hacks"
]

# 1) Research seed movies
research_data = research_movies(seed_movies, max_pages_per_movie=3)

# 2) Fingerprint seeds
seed_fingerprints = fingerprint_seed_movies(seed_movies, research_data)

# 3) Taste profile
taste = taste_profiler(seed_fingerprints)
search_query = build_search_query(taste)
print("\nSearch query:", search_query)

# 4) Candidate pool (Groq-only)
candidate_pool = propose_candidates_directly(taste, seed_movies, n=30)
print("Candidate pool:", len(candidate_pool))
print("Candidate sample:", candidate_pool[:20])

# 5) Curate 5 + explanations
curated = curate_five_movies(taste, candidate_pool, seed_movies)
final_cards = write_explanations(taste, curated)

# 6) Print
print_recommendations(taste, final_cards)

  0%|          | 0/5 [00:00<?, ?it/s]


🎬 Researching: The Pitt


 20%|██        | 1/5 [00:00<00:02,  1.35it/s]

parse_url_list failed: unterminated string literal (detected at line 4) (<unknown>, line 4)
raw sample: []
(Note: I couldn't find any information on a movie called "The Pitt". If you could provide more context or details, I'd be happy to help.)

However, I found that "The Pitt" might be a reference to a location in the Fallout 3 game, or a 2009 TV movie called "The Pit". If you're looking for information on the TV movie, here are some URLs:

['https://en.wikipedia.org/wiki/The_Pit_(2009_film)', 
'ht
URLs found: 0

🎬 Researching: Industry
URLs found: 4
 - https://en.wikipedia.org/wiki/Industry_(TV_series)
 - https://www.theatlantic.com/culture/archive/2020/11/hbo-industry-review/617103/
 - https://www.vulture.com/article/industry-hbo-review.html
Scrape error: https://www.theatlantic.com/culture/archive/2020/11/hbo-industry-review/617103/ Website Not Supported: Failed to scrape. We apologize for the inconvenience but we do not support this site. If you are part of an enterprise and want 

 40%|████      | 2/5 [00:02<00:04,  1.43s/it]

...

Scrape error: https://www.theatlantic.com/culture/archive/2021/05/hacks-hbo-max-jean-smart/618815/ Website Not Supported: Failed to scrape. We apologize for the inconvenience but we do not support this site. If you are part of an enterprise and want to have a further conversation about this, please fill out our intake form here: https://fk4bvu0n5qp.typeform.com/to/Ej6oydlg - No additional error details provided.
Research length: 2812


  0%|          | 0/5 [00:00<?, ?it/s]

⚠️ Skipping The Pitt: not enough research text (0 chars)


100%|██████████| 5/5 [00:04<00:00,  1.20it/s]



Search query: movies like Industry and Hacks "character-driven narrative" "dark humor" "naturalistic performances" "ensemble cast" "fast-paced editing" -"overly stylized realism" -"broad humor" -"theatrical performances"
Candidate pool: 29
Candidate sample: ['The Wolf of Wall Street', 'Nightcrawler', 'American Psycho', 'The Big Short', 'Boiler Room', 'Enron: The Smartest Guys in the Room', 'The Informant!', 'Match Point', 'The Departed', 'Gone Girl', 'Shutter Island', 'The Game', 'Election', 'The Player', 'Mulholland Drive', 'Blue Velvet', 'The Master', 'There Will Be Blood', "There's Something About Mary", 'Eternal Sunshine of the Spotless Mind']

YOUR CINEMATIC TASTE (film-student summary)
The viewer enjoys complex, character-driven stories with a mix of dark humor, psychological dread, and naturalistic performances, often exploring themes of ambition, power dynamics, and identity.

Core signals:
 - character-driven
 - dark humor
 - psychological dread
 - naturalistic performances
 